## Multilabel Classification

In [192]:
from robingrad import Tensor, draw_dot
import robingrad.nn as nn
from robingrad.optim import SGD, Adam
from robingrad.state import get_parameters
from sklearn import datasets
import pandas as pd
import numpy as np
import edamame.eda as eda
import time
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

### Dataset

In [93]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
print(X.shape)
print(type(X))
print(y.shape)
print(type(y))
num_classes = len(np.unique(y))
print(num_classes)

(150, 4)
<class 'numpy.ndarray'>
(150,)
<class 'numpy.ndarray'>
3


In [94]:
X = pd.DataFrame(X, columns=iris.feature_names)
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [95]:
y = pd.Series(y)
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [96]:
X_train, y_train, X_test, y_test = eda.setup(X,y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [97]:
X_train_s = eda.scaling(X_train, minmaxscaler=True)
X_test_s = eda.scaling(X_test, minmaxscaler=True)
y_train = y_train.squeeze().to_numpy()
y_test = y_test.squeeze().to_numpy()
print(X_train_s.shape)
print(X_test_s.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [98]:
y_train_ohe = np.eye(num_classes)[y_train]
y_train_ohe[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [100]:
X_train_t = Tensor(X_train_s, requires_grad=True)
print(type(X_train_t), X_train_t.shape)
y_train_ohe_t = Tensor(y_train_ohe, requires_grad=True).reshape((112,3))
print(type(y_train_ohe_t), y_train_ohe_t.shape)

<class 'robingrad.tensor.Tensor'> (112, 4)
<class 'robingrad.tensor.Tensor'> (112, 3)


In [101]:
X_train_t[1]

Tensor: slice
data: 
[0.2647059  0.95454544 0.07142857 0.        ]
grad: 
[0. 0. 0. 0.]
dtype: float32

In [204]:
class RobinNet:
    def __init__(self):
        self.l1 = nn.Linear(4, 16)
        self.l2 = nn.Linear(16, 16)
        self.l3 = nn.Linear(16, 3)
    def __call__(self, x):
        x = self.l1(x)
        x = x.relu()
        x = self.l2(x)
        x = x.relu()
        x = self.l3(x)

        return x

In [205]:
model = RobinNet()
params = get_parameters(model)
opt = SGD(params)

In [206]:
def cross_entropy_loss(forward_array, target):
    exp_forward = forward_array.exp()
    norm_term = Tensor.full_like(exp_forward, fill_value=exp_forward.sum().data, requires_grad=True)
    softmax_vec = exp_forward/norm_term
    neg_output = 1 - softmax_vec
    try:
        bce_losses = -(target * softmax_vec.log() + (1-target)*neg_output.log())
        BCELoss = bce_losses.sum()
        y_pred = softmax_vec.data.tolist()[0].index(max(softmax_vec.data.tolist()[0]))
        return BCELoss, y_pred
    except:
        print(f"forward_array: {forward_array.data}")
        print(f"exp_forward: {exp_forward.data}")
        print(f"norm_term: {norm_term.data}")
        print(f"softmax_vec: {softmax_vec.data}")
        print(f"neg_output: {neg_output.data}")

In [207]:
def accuracy_val(y_pred, y_true):
    y_pred = np.array(y_pred).round()
    correct_results = np.sum(y_pred == y_true)
    acc = correct_results/y_true.shape[0]
    acc = np.round(acc * 100)
    return acc

In [208]:
epochs = 100
for epoch in range(epochs):
    y_pred = []
    losses = []
    s = time.monotonic()
    for i in range(X_train_t.shape[0]):
        output = model(X_train_t[i].reshape((1,4)))
        loss, y_pred_i = cross_entropy_loss(output, y_train_ohe_t[i].reshape((1,3)))
        losses.append(loss.data)
        y_pred.append(y_pred_i)
        opt.zero_grad()
        loss.backward()
        opt.step()
    e = time.monotonic()
    t = e - s
    acc = accuracy_val(y_pred=y_pred, y_true=y_train)
    loss_epoch = sum(losses)/len(losses)
    if epoch % 10 == 0 or epoch==(epochs-1):
        print(f"epoch: {epoch} | loss: {loss_epoch:.2f} | time: {t:.2f} sec. | acc: {acc:.2f}%")

epoch: 0 | loss: 2.32 | time: 0.09 sec. | acc: 43.00%
epoch: 10 | loss: 0.60 | time: 0.08 sec. | acc: 90.00%
epoch: 20 | loss: 0.59 | time: 0.07 sec. | acc: 88.00%
epoch: 30 | loss: 0.37 | time: 0.07 sec. | acc: 96.00%
forward_array: [[-5.0335493e+03 -5.0377456e+03 -3.0000003e+16]]
exp_forward: [[0. 0. 0.]]
norm_term: [[0. 0. 0.]]
softmax_vec: [[nan nan nan]]
neg_output: [[nan nan nan]]


/Users/marcosalvalaggio/opt/anaconda3/lib/python3.9/site-packages/robingrad/tensor.py:164: RuntimeWarning: overflow encountered in multiply
  other.grad += np.sum(self.data * out.grad, axis=0) if other.data.shape != out.data.shape else (self.data * out.grad)
/Users/marcosalvalaggio/opt/anaconda3/lib/python3.9/site-packages/robingrad/tensor.py:174: RuntimeWarning: invalid value encountered in multiply
  self.grad += (other * self.data**(other-1)) * out.grad
/Users/marcosalvalaggio/opt/anaconda3/lib/python3.9/site-packages/robingrad/tensor.py:171: RuntimeWarning: divide by zero encountered in reciprocal
  out = Tensor(self.data ** other, dtype=self.data.dtype, _children=(self,), _op=f'**{other}', _origin="__pow__", requires_grad=self.requires_grad)
/Users/marcosalvalaggio/opt/anaconda3/lib/python3.9/site-packages/robingrad/tensor.py:158: RuntimeWarning: invalid value encountered in multiply
  out = Tensor(self.data * other.data, dtype=self.data.dtype, _children=(self, other), _op='*', _o

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
model = RobinNet()
params = get_parameters(model)
opt = SGD(params)

In [ ]:
index = 25
output = model(X_train_t[index].reshape((1,4)))
print(f"output: {output.data}")
target = y_train_ohe_t[index].reshape((1,3))
print(f"target: {target.data}")
y_true = y_train[index]
print(f"y_true: {y_true}")
def cross_entropy_loss(forward_array, target):
    print(f"forward_array: {forward_array}")
    exp_forward = forward_array.exp()
    print(f"exp_forward: {exp_forward.data}")
    norm_term = Tensor.full_like(exp_forward, fill_value=exp_forward.sum().data, requires_grad=True)
    print(f"norm_term: {norm_term.data}")
    softmax_vec = exp_forward/norm_term
    print(f"softmax_vec: {softmax_vec.data}")
    neg_output = 1 - softmax_vec
    print(f"neg_output: {neg_output.data}")
    bce_losses = -(target * softmax_vec.log() + (1-target)*neg_output.log())
    print(f"bce_losses: {bce_losses.data}")
    BCELoss = bce_losses.sum()
    print(f"BCELoss: {BCELoss.data}")
    y_pred = softmax_vec.data.tolist()[0].index(max(softmax_vec.data.tolist()[0]))
    return BCELoss, y_pred
loss, pred = cross_entropy_loss(output, target)
opt.zero_grad()
loss.backward()
opt.step()
print(pred)